In [23]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df=pd.read_csv(r"C:\Users\nice\Desktop\projects\internship system recomdation\data\Clean_content_based.csv")

In [25]:
df.head(1)

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary_raw,Text_raw,Text_clean,Summary_clean
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...,good quality dog food


In [26]:
df=df.dropna()

In [27]:
df["text_all"] = df["Text_clean"] + " " + df["Summary_clean"]
product_profiles = df.groupby("ProductId")["text_all"].apply(lambda x: " ".join(x)).reset_index()
product_profiles

,ProductId,text_all
0,0006641040,day person say chicken soup probably going fol...
1,141278509X,product archer farm best drink mix ever mix fl...
2,2734888454,dog love chicken product china wont buying any...
3,2841233731,book easy read ingredient available store unli...
4,7310172001,product health snack pup made beef liver puppy...
...,...,...
74051,B009UOFTUI,coffee supposedly premium taste watery thin no...
74052,B009UOFU20,coffee supposedly premium taste watery thin no...
74053,B009UUS05I,basket perfect sympathy gift trying express mu...
74054,B009WSNWC4,purchased product local store kid love quick e...


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=20000, ngram_range=(1,2))
tfidf_matrix=tfidf.fit_transform(product_profiles["text_all"])
import joblib
joblib.dump(tfidf, r"C:\Users\nice\Desktop\projects\internship system recomdation\models\model2.pkl")

['C:\\Users\\nice\\Desktop\\projects\\internship system recomdation\\models\\model2.pkl']

In [29]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(metric="cosine", algorithm="brute")
nn.fit(tfidf_matrix)
distances, indices = nn.kneighbors(tfidf_matrix, n_neighbors=20)

In [30]:
product_id=product_profiles["ProductId"].tolist()
product_id

['0006641040',
 '141278509X',
 '2734888454',
 '2841233731',
 '7310172001',
 '7310172101',
 '7800648702',
 '9376674501',
 'B00002N8SM',
 'B00002NCJC',
 'B00002Z754',
 'B00004CI84',
 'B00004CXX9',
 'B00004RAMS',
 'B00004RAMV',
 'B00004RAMX',
 'B00004RAMY',
 'B00004RBDU',
 'B00004RBDW',
 'B00004RBDZ',
 'B00004RYGX',
 'B00004S1C5',
 'B00004S1C6',
 'B000052Y74',
 'B00005344V',
 'B0000535LF',
 'B0000537KC',
 'B00005C2JE',
 'B00005C2M2',
 'B00005C2M3',
 'B00005IX96',
 'B00005IX97',
 'B00005IX98',
 'B00005OMWO',
 'B00005OMWQ',
 'B00005U2FA',
 'B00005V3DC',
 'B0000643LM',
 'B0000691JF',
 'B000069KQW',
 'B00006FMLY',
 'B00006G930',
 'B00006IDJO',
 'B00006IDJU',
 'B00006IDK9',
 'B00006L2ZT',
 'B00006LL38',
 'B00008433V',
 'B000084346',
 'B00008434E',
 'B00008434F',
 'B00008437P',
 'B000084388',
 'B000084DVR',
 'B000084DWM',
 'B000084DXS',
 'B000084DYL',
 'B000084E1U',
 'B000084E66',
 'B000084E6V',
 'B000084E6Y',
 'B000084E76',
 'B000084E7R',
 'B000084E9J',
 'B000084E9M',
 'B000084EA6',
 'B000084E

In [31]:
import numpy as np
np.save(r"C:\Users\nice\Desktop\projects\internship system recomdation\metrics\indices.npy", indices)
np.save(r"C:\Users\nice\Desktop\projects\internship system recomdation\metrics\distances.npy", distances)
np.save(r"C:\Users\nice\Desktop\projects\internship system recomdation\metrics\proudict_id.npy",product_id)

In [32]:
def get_recommendations(product_id, product_ids, indices, top_n=10):
    product_idx = product_ids.index(product_id)
    similar_indices = indices[product_idx][1:top_n+1]
    return [product_ids[i] for i in similar_indices]

In [33]:
product_ids = product_profiles["ProductId"].tolist()
sample_product = df["ProductId"][100] 
print("منتج الإدخال:", sample_product)
recommended = get_recommendations(sample_product, product_ids, indices, top_n=10)
print("المنتجات المقترحة:", recommended)

منتج الإدخال: B004K2IHUO
المنتجات المقترحة: ['B006L4FMR0', 'B00815FR68', 'B002Y1CVZU', 'B000PKMN14', 'B006L4FNB0', 'B004K2IHUO', 'B0007SNZGG', 'B000WY9JS8', 'B0019413WA', 'B000F9ZM8G']
